# Laboratorio no. 3 Cifrado de Información
Marco Jurado 20308

In [2]:
from Crypto.Cipher import DES, DES3, AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from PIL import Image
import numpy as np
import math

## Problema 1

AES con CBC

In [3]:
def aes_cbc_encrypt(x, y):
    cipher = AES.new(y, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(x, AES.block_size))
    iv = cipher.iv
    return iv, ct_bytes

def aes_cbc_decrypt(iv, ct_bytes, y):
    cipher = AES.new(y, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct_bytes), AES.block_size)
    return pt

AES con ECB

In [4]:
def aes_ecb_encrypt(x, y):
    cipher = AES.new(y, AES.MODE_ECB)
    ct_bytes = cipher.encrypt(pad(x, AES.block_size))
    return ct_bytes

def aes_ecb_decrypt(ct_bytes, y):
    cipher = AES.new(y, AES.MODE_ECB)
    pt = unpad(cipher.decrypt(ct_bytes), AES.block_size)
    return pt

Pipeline para cifrado y descifrado AES con ECB y CBC

In [ ]:
def encrypt_image(x, y, modo_cifrado=AES.MODE_CBC):
    img = Image.open(x).convert("RGB")
    bytes_de_la_imagen = img.tobytes()
    ancho, alto = img.size

    if modo_cifrado == AES.MODE_CBC:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(y, modo_cifrado, iv)
    else:
        cipher = AES.new(y, modo_cifrado)
        iv = None

    datos_encriptados = cipher.encrypt(pad(bytes_de_la_imagen, AES.block_size))

    tempLargo = len(datos_encriptados)
    new_alto = math.ceil(tempLargo / (3 * ancho))
    if new_alto * ancho * 3 > tempLargo:
        datos_encriptados += b'\x00' * (new_alto * ancho * 3 - tempLargo)

    return Image.fromarray(np.frombuffer(datos_encriptados, dtype=np.uint8).reshape((new_alto, ancho, 3)), 'RGB')

def decrypt_image(x, y, modo_cifrado, tamano, iv=None):
    if modo_cifrado == AES.MODE_CBC and iv is not None:
        cipher = AES.new(y, modo_cifrado, iv)
    else:
        cipher = AES.new(y, modo_cifrado)

    return Image.frombytes("RGB", tamano, unpad(cipher.decrypt(x), AES.block_size))

def encrypt_image_data(image_path, key, mode=AES.MODE_CBC):
    img = Image.open(image_path).convert("RGB")
    bytes_de_la_imagen = img.tobytes()

    if mode == AES.MODE_CBC:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key, mode, iv)
    else:
        cipher = AES.new(key, mode)
        iv = None

    datos_encriptados = cipher.encrypt(pad(bytes_de_la_imagen, AES.block_size))

    return datos_encriptados, iv, img.size

Descifrando ambas imagenes con ECB

Descifrando ambas imagenes con CBC